In [1]:
import json
import os
import argparse
import sys

import torch
from torch import nn, optim
from torch.nn import functional as F
from CAE import Convolution_Auto_Encoder as model

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(""))))
import commons
import utils
from utils import load_filepaths_and_text, load_wav_to_torch

In [2]:
def audio_processing(audio_path, hps):
    audio, sr = load_wav_to_torch(audio_path)
    audio_norm = audio/ hps.data.max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    stft = commons.TacotronSTFT(hps.data.filter_length, hps.data.hop_length, 
                               hps.data.win_length, hps.data.n_mel_channels, 
                               hps.data.sampling_rate, hps.data.mel_fmin,
                               hps.data.mel_fmax)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = torch.squeeze(torch.tensor(melspec),0)
    melspec = torch.abs(melspec)
    melspec = melspec.unsqueeze(dim = 0)
    
    frames = int(hps.data.sampling_rate / hps.data.win_length * (hps.data.win_length / hps.data.hop_length) * hps.data.slice_length)
    mel_padded = torch.FloatTensor(1, hps.data.n_mel_channels, frames)
    mel_padded.zero_()
    if melspec.size(2) < frames:
        mel_padded[:, :, :melspec.size(2)] = melspec
    else : 
        mel_padded[:, :, :frames] = melspec[:, :, :frames]
    return mel_padded*0.1

In [3]:
saved_model_path = "/media/caijb/data_drive/autoencoder/log/kernel5"
config_path = os.path.join(saved_model_path, "config.json")
with open(config_path,"r") as f:
        data = f.read()
config = json.loads(data)
hps = utils.HParams(**config)


checkpoint_path = utils.latest_checkpoint_path(saved_model_path)

CAC = model(encoder_dim=hps.model.encoder_dim, hidden_1dim=hps.model.hidden_dim1, kernel=hps.model.kernel)

CAC, _, _, _ = utils.load_checkpoint(checkpoint_path, CAC)

/media/caijb/data_drive/autoencoder/log/kernel5/G_500.pth
INFO:root:Loaded checkpoint '/media/caijb/data_drive/autoencoder/log/kernel5/G_500.pth' (iteration 500)


In [4]:
text_path = "/media/caijb/data_drive/data/Zeroshot/vctk_test.txt"
with open(text_path, "r") as f:
    data = f.read()
    data = data.split("\n")

    
result = []
for i in data:
    i = i.split("|")
    audio_path, sid = i[0], i[1]
    print(audio_path)
    mel = audio_processing(audio_path, hps)
    vector = CAC.get_vector(mel)
    c,w,h = vector.size()
    vector = vector.view(c*w*h)
    temp_vector = vector.tolist()
    temp_vector.append(int(sid))
    result.append(temp_vector)


/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_009.wav


/home/caijb/Desktop/zero_shot_glowtts/stft.py:68: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/home/caijb/Desktop/zero_shot_glowtts/commons.py:172: FutureWarning: Pass sr=22050, n_fft=1024, n_mels=80, fmin=0.0, fmax=8000.0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  sampling_rate, filter_length, n_mel_channels, mel_fmin, mel_fmax)
/home/caijb/anaconda3/envs/pytts/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_010.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_020.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_021.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_030.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_033.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_046.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_049.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_060.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_061.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_072.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_073.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_094.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_103.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p225/p225_116.wav
/media/caijb/data_dr

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_014.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_023.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_024.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_033.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_034.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_043.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_044.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_053.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_054.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_063.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_064.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_073.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_074.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p227/p227_083.wav
/media/caijb/data_dr

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_217.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_226.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_227.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_236.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_237.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_246.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_247.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_256.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_257.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_266.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_267.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_277.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_278.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p228/p228_287.wav
/media/caijb/data_dr

...

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_370.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_379.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_380.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_389.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_390.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_399.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_400.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_409.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_410.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_419.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p351/p351_420.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p360/p360_005.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p360/p360_006.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p360/p360_015.wav
/media/caijb/data_dr

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_112.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_121.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_122.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_132.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_133.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_142.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_143.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_152.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_153.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_162.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_163.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_172.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_173.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p361/p361_182.wav
/media/caijb/data_dr

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_280.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_289.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_290.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_299.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_300.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_309.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_310.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_319.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_320.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_329.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_330.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_339.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_340.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p362/p362_349.wav
/media/caijb/data_dr

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_027.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_036.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_037.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_046.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_047.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_057.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_058.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_067.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_068.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_077.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_078.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_088.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_089.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p364/p364_099.wav
/media/caijb/data_dr

/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_316.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_325.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_326.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_335.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_336.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_345.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_346.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_355.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_356.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_365.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_366.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_375.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_376.wav
/media/caijb/data_drive/data/vctk/VCTK-Corpus/wav48/p374/p374_385.wav
/media/caijb/data_dr

In [ ]:
import pandas as pd

df = pd.DataFrame(result)
df.to_csv("./result_seen.csv")

In [ ]:
import matplotlib.pyplot as plt
df.head()

In [ ]:
y = df[[255816]]

x = df.drop([255816],axis =1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
markers = ['o', '^', 's', 'x','p']
name = ['jing Li', 'Bart de Leeuw','Tux', 'Harri Tapani Ylilammi', 'Gilles G.Le Blanc']

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda= LinearDiscriminantAnalysis(n_components =256)
#scaler = StandardScaler()
#x_scaled = scaler.fit_transform(x)
lda.fit(x,y)
x_lda = lda.transform(x)
x_lda = pd.DataFrame(x_lda)
x_lda['target'] = y

In [ ]:
import sklearn.externals import joblib
save_model = joblib.dump(lda,'lda.pkl')

In [ ]:
fig = plt.figure()
#ax = fig.gca(projection='3d')
for i, marker in enumerate(markers) :
    x_axis = x_lda[x_lda['target']==i][0]
    y_axis = x_lda[x_lda['target']==i][1]
    plt.scatter(x_axis,y_axis, marker=marker, label = name[i])

plt.legend(loc='lower left')
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.show()